# Restore ELK Snapshot on Measurement Node

## Import

In [ ]:
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
    #fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")
import mflib
print(f"MFLib version  {mflib.__version__} " )
from mflib.mflib import MFLib

## Slice Info

In [ ]:
# Change your slice name and node name
slice_name = "MyMonitoredSlice"
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

In [ ]:
mf = MFLib(slice_name)

In [ ]:
meas_node_name = mf.measurement_node_name
try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

try:
    meas_node = slice.get_node(name=meas_node_name)
except Exception as e:
    print(f"Fail: {e}")   
#print (meas_node)

## Restore elk data using existing snapshot

#### Upload your snapshot file

In [ ]:
# Enter your elk snapshot tar file name (must be in data_transfer_service/snapshots directory)
snapshot_tar_name = "elk_backup_2.tar"


meas_node.upload_file(f"./snapshots/{snapshot_tar_name}", f'/tmp/{snapshot_tar_name}')
stdout, stderr = meas_node.execute(f'sudo mkdir /home/ubuntu/snapshots') 
stdout, stderr = meas_node.execute(f'sudo mv /tmp/{snapshot_tar_name} /home/ubuntu/snapshots/{snapshot_tar_name}') 

#### View snapshots ready to be imported into ELK

In [ ]:
stdout, stderr = meas_node.execute('ls /home/ubuntu/snapshots') 

#### Import a snapshot

In [ ]:
# Uncomment below and change name to a different file from above if you'd like to
# snapshot_tar_name = "elk_backup_2.tar"

In [ ]:
stdout, stderr = meas_node.execute('sudo tar -xvf /home/ubuntu/snapshots/elk_backup_2.tar -C /home/ubuntu/snapshots') 

#### Register the repository

In [ ]:
stdout, stderr = meas_node.execute('curl -X PUT -H "Content-Type: application/json" -d \'{ "type": "fs", "settings": { "location": "/var/lib/docker/volumes/elk_data01/_data", "compress": true } }\' http://localhost:9200/_snapshot/imported_repository')

#### View repository after registration

In [ ]:
stdout, stderr = node.execute('curl -X GET "http://localhost:9200/_cat/snapshots/imported_repository?v"')

#### Delete all indices and restore snapshot (replace snapshot name and change index list if you want to customize)

In [ ]:
stdout, stderr = node.execute('curl -X DELETE "http://localhost:9200/_all"')
stdout, stderr = node.execute('curl -X POST "http://localhost:9200/_snapshot/imported_repository/snapshot_test/_restore?pretty" -H "Content-Type: application/json" -d \'{ "indices": "*", "index_settings": { "index.number_of_replicas": 1 } }\'')

### Your data is now imported on the node.

### Check existing snapshot in the repository

In [ ]:
repo_name="backup_repository"
cmd = f"curl -X GET 'http://localhost:9200?v'"
stdout, stderr= meas_node.execute(cmd)

### Check the snapshot

In [ ]:
snapshot_name="snapshot_test"
cmd = f'curl -X GET "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}?pretty"'
stdout, stderr= meas_node.execute(cmd)

In [ ]:
# EDIT THIS - choose a list of index files from your local storage to import
local_files = ["./snapshots/elk_backup_2023-06-02_18:23:33.tar"]

output = mf.upload_service_files("elk", local_files)

### Restore the data using specific snapshot

In [ ]:
# Need to specify indice names to restore, find in above cell and change below
indice_str= f'"indices": "filebeat-7.13.2-2023.05.29-000001,metricbeat-7.13.2-2023.05.29-000001,packetbeat-7.13.2-2023.05.29-000001",'
repo_name="backup_repository"
snapshot_name = "snapshot_test"
cmd = f'curl -X POST "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}/_restore?pretty" -H "Content-Type: application/json" -d \'{{{indice_str} "rename_pattern": "(.+)", "rename_replacement": "restored-$1", "include_aliases": false}}\''
#print (cmd)
stdout, stderr= meas_node.execute(cmd)

### Check the progress

In [ ]:
cmd = f'curl -XGET "http://localhost:9200/_cat/recovery?v"'
stdout, stderr= meas_node.execute(cmd)

## Once finished, check indice

In [ ]:
cmd= f'curl -XGET "http://localhost:9200/_cat/indices?pretty"'
stdout, stderr= meas_node.execute(cmd)

## Remove the index life cycle policy

In [ ]:
restored_filebeat_index="restored-filebeat-7.13.2-2023.05.29-000001"
restored_metricbeat_index="restored-metricbeat-7.13.2-2023.05.29-000001"
restored_packetbeat_index = "restored-packetbeat-7.13.2-2023.05.29-000001"
cmd1 = f'curl -X POST "http://localhost:9200/{restored_filebeat_index}/_ilm/remove?pretty"'
cmd2 = f'curl -X POST "http://localhost:9200/{restored_metricbeat_index}/_ilm/remove?pretty"'
cmd3 = f'curl -X POST "http://localhost:9200/{restored_packetbeat_index}/_ilm/remove?pretty"'
stdout, stderr= meas_node.execute(cmd1)
stdout, stderr= meas_node.execute(cmd2)
stdout, stderr= meas_node.execute(cmd3)

## Set up ssh tunnel to view the data

In [ ]:
port = "10050"
cmd=f'ssh -L {port}:localhost:5601 -F ssh_config -i slice_key ubuntu@{meas_node.get_management_ip()}'
print (f"Use [{cmd}] to set up ssh tunnel and then go to http://localhost:{port} to view the data ")

### You will see (restored) filebeat/metricbeat/packetbeat indice(6 total). The restored ones are static with no life cycle policies. The regular filebeat/metricbeat/packetbeat indice will be continuous written to by the beats.   